---
title: ASE教程
authors: 庄永斌

---

# ASE 教程

官网: https://wiki.fysik.dtu.dk/ase/index.html

ASE是建模和软件使用的python包，一旦使用ASE将我们平时的常规建模脚本化之后可以极大提升工作效率。

### 了解和查找重要的模块
在官网上左侧的菜单中找到`Modules`

<img src="figures/ase-fig01.png" width=200px>

显示页面中的上侧有常用的模块

<img src="figures/ase-fig02.png" width=400px>

In [1]:
import numpy as np

## 基础

### 文件导入
使用ASE的前提是构建或者导入分子结构。

ASE 支持多种文件格式的读取，常用的有`xyz`，`lammps-data`， `cif`。

In [2]:
#import read function
from ase.io import read, write

In [3]:
## use ?? in notebook can view the python function directly
read??

Signature: read(filename, index=None, format=None, parallel=True, **kwargs)
Source:   
def read(filename, index=None, format=None, parallel=True, **kwargs):
    """Read Atoms object(s) from file.

    filename: str or file
        Name of the file to read from or a file descriptor.
    index: int, slice or str
        The last configuration will be returned by default.  Examples:

            * ``index=0``: first configuration
            * ``index=-2``: second to last
            * ``index=':'`` or ``index=slice(None)``: all
            * ``index='-3:`` or ``index=slice(-3, None)``: three last
            * ``index='::2`` or ``index=slice(0, None, 2)``: even
            * ``index='1::2`` or ``index=slice(1, None, 2)``: odd
    format: str
        Used to specify the file-format.  If not given, the
        file-format will be guessed by the *filetype* function.
    parallel: bool
        Default is to read on master and broadcast to slaves.  Use
        parallel=False to read on all sl

使用`read`函数进行文件的读取。
读入时如不指定格式，函数会从文件扩展名来推测的文件格式。

In [4]:
filepath = './bulk_ICSD/SnO2.cif'
lattice=read(filepath)

这样我们就完成了ASE结构的读入，之后只需要操作对应的对象就可以了。

#### Atom 和 Atoms Object
Atom和Atoms是ASE的两个基本Object， Atoms由Atom构成。

In [5]:
#check object type 
print(lattice)

Atoms(symbols='Sn2O4', pbc=True, cell=[4.738, 4.738, 3.1865])


本质上Atoms是Atom的list，可以使用标序的方式来查看Atom

In [6]:
#check the atom in atoms obejct, index
print(lattice[0])
print(lattice[1])
print(lattice[2])
print(lattice[3])
print(lattice[4])
print(lattice[5])

Atom('Sn', [2.369, 2.369, 1.59325], index=0)
Atom('Sn', [0.0, 0.0, 0.0], index=1)
Atom('O', [0.9139602000000001, 3.8240398, 1.59325], index=2)
Atom('O', [3.8240398, 0.9139602000000001, 1.59325], index=3)
Atom('O', [1.4550398, 1.4550398, 0.0], index=4)
Atom('O', [3.2829602000000007, 3.2829602000000007, 0.0], index=5)


可以使用一些的method对Atom和Atoms Object的信息进行提取。可用的[Atoms Method一览](https://wiki.fysik.dtu.dk/ase/ase/atoms.html?highlight=atoms)，可用的[Atom Method一览](https://wiki.fysik.dtu.dk/ase/ase/atom.html?highlight=atom#module-ase.atom)。

#### 练习
    1.从Atoms Object中获得SnO_2晶胞边长和角度。
    2.获取SnO_2所有氧原子的z坐标，并放入一个list里。

### 可视化

ASE可以直接在Jupyter Notebook里进行可视化

In [7]:
from ase.visualize import view

In [ ]:
view??

In [8]:
#select viewer as ngl
view(lattice, viewer="ngl")

HBox(children=(NGLWidget(), VBox(children=(Dropdown(description='Show', options=('All', 'Sn', 'O'), value='All'), Dropdown(description='Color scheme', options=(' ', 'picking', 'random', 'uniform', 'atomindex', 'residueindex', 'chainindex', 'modelindex', 'sstruc', 'element', 'resname', 'bfactor', 'hydrophobicity', 'value', 'volume', 'occupancy'), value=' '), FloatSlider(value=0.8, description='Ball size', max=1.5, step=0.01)))))

### 原子的增减

#### 复制体系

Atoms Object `A`可以直接赋值到新的变量上`B`，但是改变`B`同时也会改变`A`，`B`相当于只是`A`的别名。为了重新复制一个新的Atoms Object，必须采用`copy()`函数

In [15]:
lattice_2 = lattice.copy()

In [14]:
filepath = './bulk_ICSD/SnO2.cif'
lattice=read(filepath)

#### 删除原子

In [33]:
print(lattice_2)

Atoms(symbols='Sn2O4', pbc=True, cell=[4.738, 4.738, 3.1865])


使用序号进行删除，以下是删除了序号为0的原子

In [16]:
del lattice_2[0]

In [17]:
print(lattice_2)

Atoms(symbols='SnO4', pbc=True, cell=[4.738, 4.738, 3.1865])


In [18]:
view(lattice_2)

#### 增加原子

使用`Atom`创建新的原子

In [19]:
from ase import Atom

In [20]:
oxygen = Atom('O')

In [22]:
lattice_2.extend(oxygen)

In [25]:
print(lattice_2)

Atoms(symbols='SnO5', pbc=True, cell=[4.738, 4.738, 3.1865])


### 寻找相邻原子

In [27]:
from ase.neighborlist import neighbor_list

用法`neighbor_list(quantity, Atoms, cutoff)`。
cutoff可以选择原子对和半径，quantity可以有多种选择一选中心原子`i`,周围原子`j`。
选`i`时返回的数组跟选`j`时返回的是一一对应的。
例如如下第一个是中心原子0，他的周围原子是4。以此类推。
通过numpy的bincount可以求出配位数

In [28]:
neighbor_list('i', lattice, {('Sn', 'O'): 2.2})

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5,
       5, 5])

In [29]:
neighbor_list('j', lattice, {('Sn', 'O'): 2.2})

array([4, 5, 5, 2, 3, 4, 4, 5, 3, 2, 2, 3, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0])

In [30]:
import numpy as np

0,1号原子配位数是6,而2,3,4,5号原子配位数是三

In [31]:
np.bincount(neighbor_list('i', lattice, {('Sn', 'O'): 2.2}))

array([6, 6, 3, 3, 3, 3])

### 切面

In [32]:
from ase.build import surface
TiO2=read('./bulk_ICSD/Rutile.cif')

使用`surface`函数进行切面，基本语法是surface(Atoms, Miller_index, layer_number, vacuum)。注意真空添加是slab两头都添加指定的长度。

In [33]:
slab = surface(TiO2, (1, 1, 0), 4, vacuum=5)

In [35]:
slab.get_cell_lengths_and_angles()

array([ 6.49548289,  2.959     , 21.72499617, 90.        , 90.        ,
       90.        ])

In [36]:
slab = slab * (2, 4, 1)

In [37]:
view(slab)

通过surface函数切出来的slab模型还是有点令人不满足。如果我们想要两面对称的slab就需要进行一些简单的修饰，即原子的增减

#### 练习:

    1. 我们已经切好了slab模型，请实现一个能够找出该slab模型所有原子配位数的函数。
    2. 实现一个函数，该函数能够返回该slab模型中配位数是5的Ti原子的序号。

### 调整Slab的层数

In [38]:
def del_top_layer(slab, tolerance ):
    """
    function for delete the top layer atoms
    """
    tmp = slab.copy()
    z_list = tmp.get_positions().T[2]
    z_max = z_list.max()
    del_list = [atom.index for atom in tmp if ((atom.position[2] < z_max+tolerance) 
                                                    and (atom.position[2] > z_max-tolerance))]

    del tmp[del_list]
    return tmp

def del_bottom_layer(slab, tolerance):
    """
    function for delete the top layer atoms
    """
    tmp = slab.copy()
    z_list = tmp.get_positions().T[2]
    z_min = z_list.min()
    del_list = [atom.index for atom in tmp if ((atom.position[2] < z_min+tolerance) 
                                                    and (atom.position[2] > z_min-tolerance))]

    del tmp[del_list]
    return tmp

In [39]:
slab = del_top_layer(slab, 0.05)

In [61]:
slab = del_bottom_layer(slab, 0.05)

In [40]:
view(slab)

### 添加吸附物

In [41]:
from ase.build import molecule

创建一个简单的分子比如：羟基OH

In [56]:
OH = molecule('OH')

In [52]:
OH.get_positions()

array([[ 0.      ,  0.      ,  0.108786],
       [ 0.      ,  0.      , -0.870284]])

In [55]:
view(OH)

此时创建的OH是O-H朝向z的负方向, 可以通过rotate函数来旋转吸附物。比如如下我们以x轴为中心，默认氧原子为中心点，旋转120度。如果要实现更多复杂的转动可以配合旋转轴`x, y, z`来一步一步微调，或者直接指定旋转的方向。

#### 通过旋转轴转动

In [54]:
OH.rotate(120, 'x')

In [69]:
OH.get_positions()

array([[ 0.        , -0.09421144, -0.054393  ],
       [ 0.        ,  0.75368805,  0.435142  ]])

#### 转动到指定向量方向

In [57]:
old_vec=OH[1].position - OH[0].position

In [58]:
new_vec=np.array([1.5, 1, 1])

In [59]:
OH.set_cell([10, 10, 10])

In [62]:
view(OH)

In [61]:
OH.rotate(old_vec, new_vec)

添加吸附物可以通过add_adsorbate函数来实现，但是只能够添加在上表面(+z方向)。

In [63]:
from ase.build import add_adsorbate

In [64]:
view(slab)

In [67]:
position = slab[65].position[:2]

In [68]:
position

array([3.24774145, 5.918     ])

In [69]:
add_adsorbate(slab, OH,position=position, height = 2.0)

In [70]:
view(slab)

### 自己定义吸附物添加

In [71]:
def add_top_OH(slab, site, height, site_type="ontop"):
    """
    function to add OH adsorbate on the top surface
    add function to add in the bridge site
    """
    from ase.build import molecule
    # create OH and rotate to proper position
    OH = molecule('OH')
    OH.rotate(150, 'y', center = OH[0].position)
    if site_type == "ontop":
        OH.translate(slab[site].position + [0, 0, height] - OH.positions[0])
        slab.extend(OH)
    elif site_type == "bridge":
        OH.translate((slab[site[0]].position + slab[site[1]].position)/2 + [0, 0, height] - OH.positions[0])
        slab.extend(OH)
    else:
        print("The site type {0} is not implemented yet".format(site_type))

def add_bottom_OH(slab, site, height, site_type="ontop"):
    """
    function to add OH adsorbate on the bottom surface
    add function to add in the bridge site
    """
    from ase.build import molecule
    # create OH and rotate to proper position
    OH = molecule('OH')
    OH.rotate(30, 'y', center = OH[0].position)
    if site_type == "ontop":
        OH.translate(slab[site].position + [0, 0, -height] - OH.positions[0])
        slab.extend(OH)
    elif site_type == "bridge":
        OH.translate((slab[site[0]].position + slab[site[1]].position)/2 + [0, 0, -height] - OH.positions[0])
        slab.extend(OH)
    else:
        print("The site type {0} is not implemented yet".format(site_type))

In [74]:
view(slab)

In [73]:
add_top_OH(slab, 134, 1.5)
add_bottom_OH(slab, 139, 1.5)

## 创建极化子初始结构

<img src="./figures/polaron.jpg">

极化子即Polaron，是材料中一种特殊的现象。来源于lattice跟electron的相互作用，即phonon - eletron interaction。此时极化子的能级对应的波函数是一个空间局域的波函数。同时该空间周围空间原子结构，也会较理想晶体有所不同。

计算中，要找到这样的一个波函数是不容易的。就需要对于结构优化时的初始结构做一些调整。简单的方法就是预估要localize到那个原子，然后对原子周围的配位环境进行相应的扰动。

In [75]:
def create_polaron(atom_object, center_atom_number, distance, cutoff): #create polaron
    temp_list=neighbor_list('i', atom_object, cutoff)
    temp_list_2=neighbor_list('j', atom_object, cutoff)
    for i in range(len(temp_list)):
        if temp_list[i] == center_atom_number:
            neighbor_index = temp_list_2[i]
            atom_object.set_distance(center_atom_number, neighbor_index, distance, fix=0,mic=True)

In [76]:
surface=read('./tio2_dis/IS.pdb',format='xyz')

In [77]:
view(surface)

In [78]:
surface.get_distance(164,139)

2.0527437735869523

In [79]:
create_polaron(surface,139,2.2,{('Ti','O'):2.1})

In [80]:
surface.get_distance(139,164)

2.2

In [81]:
surface.get_distance(139,159)

2.1999999999999997